In [34]:
import json
import numpy as np
import pandas as pd
import os
import glob
import shutil
from tqdm import tqdm

In [110]:
json_open = open('../data/jsons/025b8a9f86d048aeb4fb008f48395920.json', 'r')
json_load = json.load(json_open)

In [49]:
df = pd.read_excel('../data/20220811-18satou_with_key.xlsx')
df[['肺','心','肝','胆','膵','脾','副腎']] = 0
df = df.set_index('key',drop=False)
df.head()

,所見,key,肺,心,肝,胆,膵,脾,副腎
key,,,,,,,,,
0b6018fa2c09459c9f9bd2e355170151,胸部CT 前回（2022/08/08）と比較しています。\n\n右胸腔内ドレーンあり。\n右...,0b6018fa2c09459c9f9bd2e355170151,0,0,0,0,0,0,0
123ed433de104b44ad97da1dc55c827a,頭部CT 前回(2022.7.28)\n\n�@後頭蓋窩、斜台背側の髄膜腫摘出術後。\n 前...,123ed433de104b44ad97da1dc55c827a,0,0,0,0,0,0,0
287de97ae7804325a702b23081f34119,肺野に活動性の炎症や明らかな腫瘤影を認めません。\n縦隔に有意なリンパ節腫大なし。\n胸水な...,287de97ae7804325a702b23081f34119,0,0,0,0,0,0,0
c086b605c69e4c90a5fa9884521fb72e,下部胸椎・腰椎部MRI 比較画像なし。\n�@下部胸椎から第1腰椎後方固定術後。\n 固定具...,c086b605c69e4c90a5fa9884521fb72e,0,0,0,0,0,0,0
3667ef3e7ef54660963d12c7767e9484,・下腹部正中でwhirl サインを認め（図１紫○）、同部位でS状結腸は狭窄（図2→）。S状結...,3667ef3e7ef54660963d12c7767e9484,0,0,0,0,0,0,0


In [79]:
df.loc['0013490f09424d909e560d1c01cc4402','所見']

'《胸部造影CT》\n前回(2022/4/19)と比較。\n肺野に転移や活動性病変を疑う所見なし。\n右肺上葉の嚢胞に変化なし。\n\n腫大リンパ節なし。\n胸水なし。\n\n冠動脈石灰化に変化なし。\n心拡大なし。\n\n右鎖骨下静脈よりCポート留置後。\n\n右第6肋骨他の骨硬化像に変化なし。'

In [102]:
df.iloc[0,0]

'胸部CT\u3000前回（2022/08/08）と比較しています。\n\n右胸腔内ドレーンあり。\n右胸腔内に軽度のairあり。\n両側胸水あり。左胸水は前回と比べ軽度増加。\n\n右中下葉に均等影を認め、内部に気管支の拡張や嚢胞性陰影を認めます。前回と比べ、均等影は軽度軽減しています。肺炎＋肺化膿症の経過を疑います。\n左肺上葉優位のすりガラス影、粒状影は前回と比べ改善傾向、舌区の嚢胞は前回と同様です。肺炎経過を疑います。\n両肺に軽度の小葉間隔壁肥厚あり。肺水腫の可能性はあります。\n有意サイズの腫大リンパ節はありません。\n心嚢液貯留あり。前回と比べ増加。\n\n気管切開チューブあり。。\nNGチューブあり。\n右内頚静脈からCVカテーテル留置。先端はSVC。'

In [119]:
json_load[0]

{'clinical_object': {'name': 'Imaging_observation',
  'tokens': ['壁', '肥厚'],
  'mention': None,
  'start_idx': 20,
  'end_idx': 21,
  'sui': None,
  'certainty_scale': 4},
 'modifiers': [{'name': 'Anatomical_entity',
   'tokens': ['胃', '体', '部', '大', '弯', '側'],
   'mention': None,
   'start_idx': 10,
   'end_idx': 15,
   'sui': None,
   'certainty_scale': None},
  {'name': 'Characteristics_descriptor',
   'tokens': ['びまん', '性'],
   'mention': None,
   'start_idx': 18,
   'end_idx': 19,
   'sui': None,
   'certainty_scale': None},
  {'name': 'Change_descriptor',
   'tokens': ['わずか', 'に', '改善', 'ある', 'か'],
   'mention': None,
   'start_idx': 23,
   'end_idx': 27,
   'sui': None,
   'certainty_scale': None}]}

In [132]:
json_lists = sorted(glob.glob('../data/jsons/*.json'))
organs = ['肝']#'心','肝','胆','膵','脾','副腎']
os.makedirs('../output',exist_ok=True)
counter = 0

def search_anatomy(modifiers,organ):
    for i in range(len(modifiers)):
        modifier = modifiers[i]
        if organ in modifier['tokens'] and modifier['name'] == 'Anatomical_entity':
            return True
    return False

def search_change(modifiers):
    for i in range(len(modifiers)):
        modifier = modifiers[i]
        if modifier['name'] == 'Change_descriptor':
            return True,''.join(modifier['tokens'])
    return False,0

def search_clinicalfinding(modifiers):
    for i in range(len(modifiers)):
        modifier = modifiers[i]
        if modifier['name'] == 'Clinical_finding':
            return True,''.join(modifier['tokens'])
    return False,0

#def detect_change(modifiers):
change_word_list = []
clinical_word_list = []

for json_list in tqdm(json_lists):
    json_open = open(json_list,'r')
    key = os.path.basename(json_list).split('.')[0]
    json_load = json.load(json_open)
    for organ in organs:
        #print(organ)
        for i in range(len(json_load)):
            bool_anatomy = search_anatomy(json_load[i]['modifiers'],organ)
            bool_change,change = search_change(json_load[i]['modifiers'])
            bool_clinical,clinical = search_clinicalfinding(json_load[i]['modifiers'])
            if bool_anatomy and bool_change and bool_clinical:
                #print(json_list,json_load[i]['modifiers'])
                #df.loc[key,organ] = str(json_load[i])
                if change not in change_word_list:
                    change_word_list.append(change)
                if clinical not in clinical_word_list:
                    clinical_word_list.append(clinical)
                counter += 1
            else:
                continue

print(counter,organ)

100%|██████████| 6636/6636 [00:00<00:00, 13529.61it/s]

255 肝


In [134]:
pd.DataFrame(clinical_word_list).to_csv('clinical_discriptor.csv',index = False)

In [128]:
pd.DataFrame(change_word_list).to_csv('change_discriptor.csv',index = False)

In [64]:
json_load[i]


{'clinical_object': {'name': 'Imaging_observation',
  'tokens': ['小', '嚢胞'],
  'mention': None,
  'start_idx': 12,
  'end_idx': 13,
  'sui': None,
  'certainty_scale': 4},
 'modifiers': [{'name': 'Anatomical_entity',
   'tokens': ['肝'],
   'mention': None,
   'start_idx': 10,
   'end_idx': 10,
   'sui': None,
   'certainty_scale': None}]}

In [72]:
df.to_csv('../output/sample.csv')

In [19]:
json_open = open('../json_data/response_1660614630854.json', 'r')
json_load = json.load(json_open)

In [20]:
json_load

[{'clinical_object': {'name': 'Imaging_observation',
   'tokens': ['小', '嚢胞'],
   'mention': '小嚢胞',
   'start_idx': 7,
   'end_idx': 8,
   'sui': 'OBJ002760',
   'certainty_scale': 4},
  'modifiers': [{'name': 'Anatomical_entity',
    'tokens': ['肝'],
    'mention': '肝',
    'start_idx': 0,
    'end_idx': 0,
    'sui': None,
    'certainty_scale': None},
   {'name': 'Anatomical_entity',
    'tokens': ['S', '７', '／', '８'],
    'mention': 'S７／８',
    'start_idx': 2,
    'end_idx': 5,
    'sui': None,
    'certainty_scale': None}]},
 {'clinical_object': {'name': 'Imaging_observation',
   'tokens': ['胆石'],
   'mention': '胆石',
   'start_idx': 20,
   'end_idx': 20,
   'sui': 'OBJ004923',
   'certainty_scale': 4},
  'modifiers': [{'name': 'Anatomical_entity',
    'tokens': ['胆嚢'],
    'mention': '胆嚢',
    'start_idx': 18,
    'end_idx': 18,
    'sui': None,
    'certainty_scale': None}]},
 {'clinical_object': {'name': 'Imaging_observation',
   'tokens': ['嚢胞'],
   'mention': '嚢胞',
   'start_i